## 01_ETL

In [76]:
# importar librerias necesarias
import pandas as pd
import geopandas as gpd

##### Importar

In [77]:
# sig data
sig_provincias = gpd.read_file("data/sig/provincia.shp")
sig_localidades = gpd.read_file("data/sig/localidad_bahra.shp")

In [78]:
# informacion a nivel nacional
velocidad_nac_por_año = pd.read_csv("data/unprocessed/historico_velocidad_internet.csv")
ingresos_nac = pd.read_csv("data/unprocessed/Internet_Ingresos.csv")
penetracion_nac = pd.read_csv("data/unprocessed/Internet_Penetracion (1).csv")
baf_nac = pd.read_csv("data/unprocessed/Internet_BAF.csv")
accesos_nac_por_tecnologia = pd.read_csv("data/unprocessed/Internet_Accesos-por-tecnologia.csv")
accesos_nac_por_velocidad = pd.read_csv("data/unprocessed/Internet_Accesos-por-velocidad nacionales transformado.csv")

In [79]:
# informacion a nivel provincial
accesos_prov_por_tecnologia = pd.read_csv("data/unprocessed/Internet_Accesos-por-tecnologia (1).csv")
accesos_prov_por_velocidad = pd.read_csv("data/unprocessed/AccesosaInternetfijoporvelocidadbajadayprovincia_transformado.csv")
penetracion_prov = pd.read_csv("data/unprocessed/Internet_Penetracion.csv")
baf_prov = pd.read_csv("data/unprocessed/Internet_BAF (1).csv")

In [80]:
# informacion a nivel local
accesos_loc_por_tecnologia = pd.read_csv("data/unprocessed/AccesosaInternetfijoportecnologiaylocalidad_2791751698918580638.csv")
accesos_loc_por_velocidad = pd.read_csv("data/unprocessed/AccesosaInternetfijoporvelocidaddebajadaylocalidad_transformado.csv")
localidades_con_conectividad = pd.read_csv("data/unprocessed/Listadodelocalidadesconconectividadainternet_2750221698918588798.csv")

##### Transformar

In [81]:
# sig data
sig_provincias.drop(columns=["gid", "entidad", "fna", "gna", "in1", "fdc", "sag"], inplace=True)
sig_provincias.rename(columns={"nam": "provincia"}, inplace=True)
sig_localidades.rename(columns={"nom_agl": "localidad", "nom_depto": "departamento", "nom_pcia": "provincia"}, inplace=True) 
sig_localidades.drop(columns=["gid", "cod_pcia", "cod_depto", "cod_ase", "fna", "cod_aglo", "nom_aglo", "cod_agl", "lat_gd", "long_gd", "lat_gs", "long_gs", "fdc", "tipo_asent"], inplace=True)

In [82]:
# informacion a nivel nacional
velocidad_nac_por_año.rename(columns={"Mbps (Media de bajada)": "Velocidad media", "Trimestre.1": "Periodo"}, inplace=True)
velocidad_nac_por_año['Velocidad media'] = velocidad_nac_por_año['Velocidad media'].apply(lambda x: x / 1000 if x > 350 else x)
ingresos_nac.rename(columns={"OTROS": "Otros"}, inplace=True)
ingresos_nac["Ingresos"] = ingresos_nac["Ingresos (miles de pesos)"] * 1000
ingresos_nac.drop(columns="Ingresos (miles de pesos)", inplace=True)
penetracion_nac["Accesos por cada 100 hogares"].replace(",", ".", regex=True, inplace=True)
penetracion_nac["Accesos por cada 100 hogares"] = penetracion_nac["Accesos por cada 100 hogares"].astype(float)
penetracion_nac["Accesos por cada 100 hab"].replace(",", ".", regex=True, inplace=True)
penetracion_nac["Accesos por cada 100 hab"] = penetracion_nac["Accesos por cada 100 hab"].astype(float)
accesos_nac_por_tecnologia = accesos_nac_por_tecnologia.merge(baf_nac[["Año", "Trimestre", "Banda ancha fija", "Dial up"]], on=["Año", "Trimestre"])
accesos_nac_por_tecnologia.rename(columns={"Dial up": "Dial Up", "Banda ancha fija": "BAF"}, inplace=True)
accesos_nac_por_velocidad.rename(columns={"OTROS": "Otros"}, inplace=True)

In [83]:
accesos_nac_por_tecnologia["ADSL"] = accesos_nac_por_tecnologia["ADSL"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Cablemodem"] = accesos_nac_por_tecnologia["Cablemodem"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Fibra óptica"] = accesos_nac_por_tecnologia["Fibra óptica"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Wireless"] = accesos_nac_por_tecnologia["Wireless"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Otros"] = accesos_nac_por_tecnologia["Otros"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Total"] = accesos_nac_por_tecnologia["Total"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["BAF"] = accesos_nac_por_tecnologia["BAF"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Dial Up"] = accesos_nac_por_tecnologia["Dial Up"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Wireless"] = accesos_nac_por_tecnologia["Wireless"].apply(lambda x: x / 1000 if x > 50000 else x)
accesos_nac_por_tecnologia["Otros"] = accesos_nac_por_tecnologia["Otros"].apply(lambda x: x / 1000 if x > 50000 else x)

In [84]:
# informacion a nivel provincial
accesos_prov_por_tecnologia["Año"].replace("2019 *", "2019", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("1 *", "1", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("2 *", "2", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("3 *", "3", inplace=True)
accesos_prov_por_tecnologia.drop(labels=864, inplace=True)
accesos_prov_por_tecnologia["Año"] = accesos_prov_por_tecnologia["Año"].astype(int)
accesos_prov_por_tecnologia["Trimestre"] = accesos_prov_por_tecnologia["Trimestre"].astype(int)
#accesos_prov_por_tecnologia = accesos_prov_por_tecnologia.merge(baf_prov[["Año", "Trimestre", "Banda ancha fija", "Dial up"]], on=["Año", "Trimestre"])
#accesos_prov_por_tecnologia.rename(columns={"Dial up": "Dial Up", "Banda ancha fija": "BAF"}, inplace=True)
accesos_prov_por_velocidad.drop(labels=0, inplace=True)
accesos_prov_por_velocidad.fillna(0, inplace=True)
penetracion_prov.drop(columns=["Unnamed: 4", "Unnamed: 5", "Unnamed: 6"], inplace=True)
penetracion_prov["Accesos por cada 100 hogares"].replace(",", ".", regex=True, inplace=True)
penetracion_prov["Accesos por cada 100 hogares"] = penetracion_prov["Accesos por cada 100 hogares"].astype(float)

In [85]:
# informacion a nivel local
accesos_loc_por_tecnologia.rename(columns={"OTROS": "Otros", "CABLEMODEM": "Cablemodem", "FIBRA OPTICA": "Fibra óptica", "SATELITAL": "Satelital", "WIMAX": "Wimax", "WIRELESS": "Wireless", "DIAL UP": "Dial Up", "Total general": "Total"}, inplace=True)
accesos_loc_por_tecnologia.drop(columns="Unnamed: 13", inplace=True)
accesos_loc_por_tecnologia["ADSL"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["ADSL"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["ADSL"] = accesos_loc_por_tecnologia["ADSL"].astype(float)
accesos_loc_por_tecnologia["Cablemodem"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Cablemodem"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Cablemodem"] = accesos_loc_por_tecnologia["Cablemodem"].astype(float)
accesos_loc_por_tecnologia["Dial Up"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Dial Up"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Dial Up"] = accesos_loc_por_tecnologia["Dial Up"].astype(float)
accesos_loc_por_tecnologia["Fibra óptica"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Fibra óptica"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Fibra óptica"] = accesos_loc_por_tecnologia["Fibra óptica"].astype(float)
accesos_loc_por_tecnologia["Otros"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Otros"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Otros"] = accesos_loc_por_tecnologia["Otros"].astype(float)
accesos_loc_por_tecnologia["Satelital"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Satelital"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Satelitala"] = accesos_loc_por_tecnologia["Satelital"].astype(float)
accesos_loc_por_tecnologia["Wimax"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wimax"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wimax"] = accesos_loc_por_tecnologia["Wimax"].astype(float)
accesos_loc_por_tecnologia["Wireless"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wireless"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wireless"] = accesos_loc_por_tecnologia["Wireless"].astype(float)
accesos_loc_por_tecnologia["Total"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Total"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Total"] = accesos_loc_por_tecnologia["Total"].astype(float)
localidades_con_conectividad.rename(columns={"Cablemódem": "Cablemodem"}, inplace=True)
localidades_con_conectividad["ADSL"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["ADSL"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Cablemodem"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Cablemodem"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Dial Up"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Dial Up"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Fibra óptica"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Fibra óptica"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["4G"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["4G"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["3G"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["3G"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Telefonía Fija"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Telefonía Fija"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Wireless"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Wireless"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Satelital"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Satelital"].replace("--", False, regex=True, inplace=True)

##### Dashboar data preparation

In [86]:
dashboard_tecnologias_por_provincia = pd.DataFrame(columns=["Año", "Trimestre", "Provincia", "Tecnologia"])

for tecnologia in ["ADSL", "Cablemodem", "Fibra óptica", "Wireless", "Otros"]:
    data = accesos_prov_por_tecnologia[["Año", "Trimestre", "Provincia", tecnologia]]
    data = data.rename(columns={tecnologia: "Conexiones"})
    data["Tecnologia"] = tecnologia
    dashboard_tecnologias_por_provincia = pd.concat([dashboard_tecnologias_por_provincia, data], ignore_index=True)

In [87]:
def asignar_zona(provincia):
    noreste = ["Formosa", "Chaco", "Santiago Del Estero", "Corrientes", "Misiones"]
    noroeste = ["Jujuy", "Salta", "Tucumán", "Catamarca"]
    centro = ["Capital Federal", "Buenos Aires", "Santa Fe", "Córdoba", "Entre Ríos"]
    oeste = ["San Juan", "Mendoza", "San Luis", "La Rioja", "La Pampa"]
    sur = ["Neuquén", "Río Negro", "Chubut", "Santa Cruz", "Tierra Del Fuego"]
    if provincia in noroeste:
        return 'Noroeste'
    elif provincia in noreste:
        return 'Noreste'
    elif provincia in centro:
        return 'Centro'
    elif provincia in oeste:
        return 'Oeste'
    elif provincia in sur:
        return 'Sur'
    else:
        return None
dashboard_tecnologias_por_provincia['Zona'] = dashboard_tecnologias_por_provincia['Provincia'].apply(asignar_zona)

In [88]:
# Rectificacion ADSL
for i, row in dashboard_tecnologias_por_provincia.iterrows():
    # Centro
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Buenos Aires" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Santa Fe":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Córdoba":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Entre Ríos":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Chubut":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Río Negro":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Santa Cruz":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Neuquén":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Tierra Del Fuego":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Salta":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Jujuy":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Catamarca":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Tucumán":
       dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Formosa":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Chaco":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Santiago Del Estero":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Corrientes":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Misiones":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "San Juan":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "Mendoza":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "San Luis":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "La Rioja":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "ADSL" and row["Provincia"] == "La Pampa":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000   

In [89]:
# Rectificacion Fibra óptica
for i, row in dashboard_tecnologias_por_provincia.iterrows():
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Buenos Aires" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Córdoba" and row["Conexiones"] < 370 and row["Año"] > 2014:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Santa Fe":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Entre Ríos" and row["Conexiones"] < 45:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Chaco" and row["Conexiones"] < 50 and row["Año"] > 2017:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Corrientes" and row["Conexiones"] < 30:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Formosa" and row["Conexiones"] < 16 and row["Año"] > 2018:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Misiones" and row["Conexiones"] < 47 and row["Año"] > 2018:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Santiago Del Estero" and row["Conexiones"] < 15:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Chubut" and row["Conexiones"] < 15:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Neuquén" and row["Conexiones"] < 30:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Río Negro" and row["Conexiones"] < 45:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Santa Cruz" and row["Conexiones"] < 20:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Tierra Del Fuego" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 100
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Tierra Del Fuego" and row["Conexiones"] == 0:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] = 500  

    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Salta" and row["Conexiones"] < 60 and row["Año"] > 2018:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Jujuy" and row["Conexiones"] < 45 and row["Año"] > 2018:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Catamarca" and row["Conexiones"]:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Tucumán" and row["Conexiones"] < 200 and row["Año"] > 2019:
       dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
       
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "Mendoza" and row["Conexiones"]:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "San Juan" and row["Conexiones"] < 20:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "La Rioja":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "La Pampa" and row["Conexiones"] < 30:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Fibra óptica" and row["Provincia"] == "San Luis" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

In [90]:
# Rectificacion Cablemodem
for i, row in dashboard_tecnologias_por_provincia.iterrows():
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Buenos Aires" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Capital Federal" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Santa Fe":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Córdoba":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Entre Ríos":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Salta":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Jujuy":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Catamarca" and row["Conexiones"] < 20:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Tucumán" and row["Conexiones"] < 25:
       dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Mendoza" and row["Conexiones"]:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "San Juan" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "La Rioja":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "La Rioja" and row["Conexiones"] == 0:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] = 35000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "La Pampa":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "San Luis" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Formosa":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Chaco":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Santiago Del Estero":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Corrientes":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Misiones":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Chubut" and row["Conexiones"] < 100:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Neuquén":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Río Negro":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Santa Cruz" and row["Conexiones"] < 25:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Cablemodem" and row["Provincia"] == "Tierra Del Fuego":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

In [91]:
# Rectificacion Wireless
for i, row in dashboard_tecnologias_por_provincia.iterrows():
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Santa Fe":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Córdoba":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Entre Ríos":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Salta" and row["Conexiones"] < 15 and row["Año"] > 2019:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Jujuy" and row["Conexiones"] < 10 and row["Año"] > 2019:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Catamarca" and row["Conexiones"] < 10 and row["Año"] > 2019:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Tucumán" and row["Conexiones"] < 10:
       dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Mendoza":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "San Juan" and row["Conexiones"] < 25:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "La Rioja" and row["Conexiones"] < 15:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "La Pampa":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "San Luis":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Formosa" and row["Conexiones"] < 20:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Formosa" and row["Conexiones"] == 0:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] = 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Chaco" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Santiago Del Estero" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Corrientes":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Misiones" and row["Conexiones"] < 20:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Chubut":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Neuquén":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Río Negro":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Wireless" and row["Provincia"] == "Santa Cruz" and row["Conexiones"] < 25:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

In [92]:
# Rectificacion Otros
for i, row in dashboard_tecnologias_por_provincia.iterrows():
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Santa Fe" and row["Conexiones"] < 25:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Córdoba" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Entre Ríos" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Salta" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Jujuy" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Catamarca" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Mendoza" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "San Juan" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "La Rioja":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "otros" and row["Provincia"] == "La Pampa" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "San Luis" and row["Conexiones"] < 50:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Chaco" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Santiago Del Estero" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Corrientes" and row["Conexiones"] < 5:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Misiones" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000

    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Chubut":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Neuquén" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Río Negro":
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Santa Cruz" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000
    if row["Tecnologia"] == "Otros" and row["Provincia"] == "Tierra Del Fuego" and row["Conexiones"] < 10:
        dashboard_tecnologias_por_provincia.loc[i, "Conexiones"] *= 1000


In [93]:
# Calcular penetracion
dashboard_tecnologias_por_provincia["Penetracion cada 100 hab"] = None

for i, row in dashboard_tecnologias_por_provincia.iterrows():
    if row["Provincia"] == "Buenos Aires":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 17569000 ) * 100
    if row["Provincia"] == "Capital Federal":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 3120000 ) * 100
    if row["Provincia"] == "Catamarca":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 396000 ) * 100    
    if row["Provincia"] == "Chaco":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1140000 ) * 100
    if row["Provincia"] == "Chubut":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 603000 ) * 100
    if row["Provincia"] == "Córdoba":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 3308000 ) * 100
    if row["Provincia"] == "Corrientes":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1197000 ) * 100
    if row["Provincia"] == "Entre Ríos":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1426000 ) * 100
    if row["Provincia"] == "Formosa":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 530000 ) * 100
    if row["Provincia"] == "Jujuy":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 797000 ) * 100
    if row["Provincia"] == "La Pampa":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 366000 ) * 100
    if row["Provincia"] == "La Rioja":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 315000 ) * 100
    if row["Provincia"] == "Mendoza":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1741000 ) * 100
    if row["Provincia"] == "Misiones":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1280000 ) * 100
    if row["Provincia"] == "Neuquén":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 726000 ) * 100
    if row["Provincia"] == "Río Negro":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 762000 ) * 100
    if row["Provincia"] == "Salta":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1440000 ) * 100
    if row["Provincia"] == "San Juan":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 818000 ) * 100
    if row["Provincia"] == "San Luis":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 540000 ) * 100
    if row["Provincia"] == "Santa Cruz":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 333000 ) * 100
    if row["Provincia"] == "Santa Fe":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 3400000 ) * 100
    if row["Provincia"] == "Santiago Del Estero":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 1054000 ) * 100
    if row["Provincia"] == "Tierra Del Fuego":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 190000 ) * 100
    if row["Provincia"] == "Tucumán":
        dashboard_tecnologias_por_provincia.loc[i, "Penetracion cada 100 hab"] = ( row["Conexiones"] / 2460000 ) * 100

dashboard_tecnologias_por_provincia["Penetracion cada 100 hab"] = dashboard_tecnologias_por_provincia["Penetracion cada 100 hab"].apply(lambda x: round(x, 2))
dashboard_tecnologias_por_provincia["Penetracion cada 100 hogares"] = dashboard_tecnologias_por_provincia["Penetracion cada 100 hab"] * 4

In [95]:
dashboards_kpis = pd.DataFrame(columns=["Año", "Trimestre", "Provincia", "Accesos por cada 100 hogares", "KPI1"])
provincias = ["Formosa", "Chaco", "Santiago Del Estero", "Corrientes", "Misiones", "Jujuy", "Salta", "Tucumán", "Catamarca", "Capital Federal", "Buenos Aires", "Santa Fe", "Córdoba", "Entre Ríos", "San Juan", "Mendoza", "San Luis", "La Rioja", "La Pampa", "Neuquén", "Río Negro", "Chubut", "Santa Cruz", "Tierra Del Fuego"]
for provincia in provincias:
    for tecnologia in ["ADSL", "Cablemodem", "Fibra óptica", "Wireless", "Otros"]:
        data_prov = dashboard_tecnologias_por_provincia[(dashboard_tecnologias_por_provincia["Provincia"] == provincia) & (dashboard_tecnologias_por_provincia["Tecnologia"] == tecnologia)].reset_index()
        data_prov["KPI1"] = 0
        data_prov["KPI2"] = 0
        for i, row in data_prov.iterrows():
            try:
                data_prov.loc[i, "KPI1"] = row["Penetracion cada 100 hogares"] - data_prov.loc[i+1, "Penetracion cada 100 hogares"]
                data_prov.loc[i, "KPI2"] = row["Penetracion cada 100 hab"] - data_prov.loc[i+1, "Penetracion cada 100 hab"]
            except:
                data_prov.loc[i, "KPI1"] = 0
                data_prov.loc[i, "KPI1"] = 0
        dashboards_kpis = pd.concat([dashboards_kpis, data_prov], ignore_index=True)

C:\Users\david\AppData\Local\Temp\ipykernel_14068\3090491400.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.7599999999999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_prov.loc[i, "KPI1"] = row["Penetracion cada 100 hogares"] - data_prov.loc[i+1, "Penetracion cada 100 hogares"]
C:\Users\david\AppData\Local\Temp\ipykernel_14068\3090491400.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.18999999999999995' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_prov.loc[i, "KPI2"] = row["Penetracion cada 100 hab"] - data_prov.loc[i+1, "Penetracion cada 100 hab"]
C:\Users\david\AppData\Local\Temp\ipykernel_14068\3090491400.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future vers

,Año,Trimestre,Provincia,Accesos por cada 100 hogares,KPI1,index,Tecnologia,Conexiones,Zona,Penetracion cada 100 hab,Penetracion cada 100 hogares,KPI2
0,2022,4,Formosa,NaN,0.00,8.0,ADSL,14000.0,Noreste,2.64,10.56,0.00
1,2022,3,Formosa,NaN,-0.76,32.0,ADSL,14000.0,Noreste,2.64,10.56,-0.19
2,2022,2,Formosa,NaN,-0.76,56.0,ADSL,15000.0,Noreste,2.83,11.32,-0.19
3,2022,1,Formosa,NaN,-0.76,80.0,ADSL,16000.0,Noreste,3.02,12.08,-0.19
4,2021,4,Formosa,NaN,-0.76,104.0,ADSL,17000.0,Noreste,3.21,12.84,-0.19
...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2015,1,Tierra Del Fuego,NaN,0.00,4222.0,Otros,3000.0,Sur,1.58,6.32,0.00
4316,2014,4,Tierra Del Fuego,NaN,0.00,4246.0,Otros,3000.0,Sur,1.58,6.32,0.00
4317,2014,3,Tierra Del Fuego,NaN,0.00,4270.0,Otros,3000.0,Sur,1.58,6.32,0.00
4318,2014,2,Tierra Del Fuego,NaN,0.00,4294.0,Otros,3000.0,Sur,1.58,6.32,0.00


##### Exportar

In [96]:
# sig datasets
sig_provincias.to_csv("data/sig_provincias.csv")
sig_localidades.to_csv("data/sig_localidades.csv")

In [97]:
# informacion a nivel nacional
velocidad_nac_por_año.to_csv("data/velocidad_nac_por_año.csv")
accesos_nac_por_tecnologia.to_csv("data/accesos_nac_por_tecnologia.csv")
accesos_nac_por_velocidad.to_csv("data/accesos_nac_por_velocidad.csv")
ingresos_nac.to_csv("data/ingresos_nac.csv")
penetracion_nac.to_csv("data/penetracion_nac.csv")

In [98]:
# informacion a nivel provincial
accesos_prov_por_tecnologia.to_csv("data/accesos_prov_por_tecnologia.csv")
accesos_prov_por_velocidad.to_csv("data/accesos_prov_por_velocidad.csv")
penetracion_prov.to_csv("data/penetracion_prov.csv")

In [99]:
# informacion a nivel local
accesos_loc_por_tecnologia.to_csv("data/accesos_loc_por_tecnologia.csv")
accesos_loc_por_velocidad.to_csv("data/accesos_loc_por_velocidad.csv")
localidades_con_conectividad.to_csv("data/localidades_con_conectividad.csv")

In [100]:
# dashboard data
dashboard_tecnologias_por_provincia["Conexiones"] = dashboard_tecnologias_por_provincia["Conexiones"].astype(int)
dashboard_tecnologias_por_provincia.to_csv("data/dashboard_tecnologias_por_provincia.csv")
dashboard_tecnologias_por_provincia.to_excel("data/dashboard_tecnologias_por_provincia.xlsx", index=False)
dashboards_kpis.to_excel("data/dashboards_kpis.xlsx", index=False)

In [101]:
dashboards_kpis

,Año,Trimestre,Provincia,Accesos por cada 100 hogares,KPI1,index,Tecnologia,Conexiones,Zona,Penetracion cada 100 hab,Penetracion cada 100 hogares,KPI2
0,2022,4,Formosa,NaN,0.00,8.0,ADSL,14000.0,Noreste,2.64,10.56,0.00
1,2022,3,Formosa,NaN,-0.76,32.0,ADSL,14000.0,Noreste,2.64,10.56,-0.19
2,2022,2,Formosa,NaN,-0.76,56.0,ADSL,15000.0,Noreste,2.83,11.32,-0.19
3,2022,1,Formosa,NaN,-0.76,80.0,ADSL,16000.0,Noreste,3.02,12.08,-0.19
4,2021,4,Formosa,NaN,-0.76,104.0,ADSL,17000.0,Noreste,3.21,12.84,-0.19
...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2015,1,Tierra Del Fuego,NaN,0.00,4222.0,Otros,3000.0,Sur,1.58,6.32,0.00
4316,2014,4,Tierra Del Fuego,NaN,0.00,4246.0,Otros,3000.0,Sur,1.58,6.32,0.00
4317,2014,3,Tierra Del Fuego,NaN,0.00,4270.0,Otros,3000.0,Sur,1.58,6.32,0.00
4318,2014,2,Tierra Del Fuego,NaN,0.00,4294.0,Otros,3000.0,Sur,1.58,6.32,0.00
